In [3]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import pandas as pd
import spacy
from spacy.util import minibatch, compounding
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from numpy import array, unique, array_equal
max_fatures = 1000


def load_data():
    print("Loading Data...")
    PrimaryEmotion = pd.read_csv('emotion.xls.csv')
    print("Data Loaded")
    return (PrimaryEmotion['sentence'], array(PrimaryEmotion['emotion']))


#Below the data is sorted into nine emotion groups. Eight of the groups are the outter layer of the wheel, or the combinations of 
#two emotion groups. The ninth group is "Ambiguous" and "Neutral" put together.
def sort_to_9_emotions(sentence_list, emotion_list):
    sorted_list = []
    sorted_emo = []
    for (data, emo) in zip(sentence_list, emotion_list):
        if (emo == 'Contempt' or emo == 'Anger' or emo == 'Disgust'):
            sorted_list.append(data)
            sorted_emo.append("Contempt")
        if (emo == 'Aggression' or emo == 'Anticipation' or emo == 'Anger'):
            sorted_list.append(data)
            sorted_emo.append("Aggression")
        if (emo == 'Optimism' or emo == 'Anticipation' or emo == 'Joy'):
            sorted_list.append(data)
            sorted_emo.append("Optimism")
        if (emo == 'Love' or emo == 'Joy' or emo == 'Trust'):
            sorted_list.append(data)
            sorted_emo.append("Love")
        if (emo == 'Submission' or emo == 'Trust' or emo == 'Fear'):
            sorted_list.append(data)
            sorted_emo.append("Submission")
        if (emo == 'Awe' or emo == 'Fear' or emo == 'Surprise'):
            sorted_list.append(data)
            sorted_emo.append("Awe")
        if (emo == 'Disapproval' or emo == 'Surprise' or emo == 'Saddness'):
            sorted_list.append(data)
            sorted_emo.append("Disapproval")
        if (emo == 'Remorse' or emo == 'Saddness' or emo == 'Disgust'):
            sorted_list.append(data)
            sorted_emo.append("Remorse")
        if (emo == 'Neutral' or emo == 'Ambiguous'):
            sorted_list.append(data)
            sorted_emo.append("Neutral")
    return (sorted_list, sorted_emo)



def tokenize(sentences):
    print("Tokenizing...")
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(sentences)
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X)
    print("Tokenization Completed")
    return X



def train(tokened_sentences, emotion_list):
    print("Training...")
    embed_dim = 128
    lstm_out = 196
    test_percent=.2

    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = tokened_sentences.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    
    # len(set(emotion_list)) is a hacky way of geting the number of unique elements
    # in a regualar python list (non-numpy)
    model.add(Dense(unique(emotion_list).size,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    #print(model.summary())

    Y = pd.get_dummies(emotion_list).values
    X_train, X_test, Y_train, Y_test = train_test_split(tokened_sentences,Y, test_size = test_percent, random_state = 42)
    #print(X_train.shape,Y_train.shape)
    #print(X_test.shape,Y_test.shape)

    batch_size = 32
    model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1)
    print("Training Completed")
    print("Testing Against Control... (% of the data) ", test_percent)
    score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
    print("Score   :", score)
    print("Accuracy:", acc)
    return (model, X_test)

### When Splitting data (train_test_split), we don't retain where in the 
### origional set the data is located, thus it takes
### a little trickery to see the results while while only testing against untrained data

def test(model, X_test, tokenized_data, sentence_list, emotion_list):
    predictions = model.predict(tokenized_data, batch_size=32)
    error = 0
    for i in range(len(predictions)):
     
        #  "If the current tokenized data array is in X_test (untrained tokenized arrays)
        
        if(any(array_equal(tokenized_data[i], x) for x in X_test)):
            pos = list(predictions[i]).index(max(predictions[i]))
            print("\n\n")
            print(sentence_list[i])
            print("\nPredicted emotion: ", unique(emotion_list)[pos])
            print("Actual emotion   : ", emotion_list[i])
            if (unique(emotion_list)[pos] != emotion_list[i]):
                error += 1
    print("%  Accuracy when against untrained set: ", 1- (float(error) / len(X_test)))

#################################
#############MAIN################
#################################
#   jupyter notebook is weird   #




(sentence_list, emotion_list) = load_data()


### Comment the Below line for all 18 emotions. This sorts into "positive" and "negative"
sentence_list, emotion_list = sort_to_9_emotions(sentence_list, emotion_list)

tokenized_data = tokenize(sentence_list)

(model, X_test) = train(tokenized_data, emotion_list)

test(model, X_test, tokenized_data, sentence_list, emotion_list)


Loading Data...
Data Loaded
Tokenizing...
Tokenization Completed
Training...
Epoch 1/10
2592/2592 [==============================] - ETA: 4:13 - loss: 2.1972 - acc: 0.093 - ETA: 2:23 - loss: 2.1949 - acc: 0.156 - ETA: 1:48 - loss: 2.1959 - acc: 0.135 - ETA: 1:28 - loss: 2.1965 - acc: 0.125 - ETA: 1:15 - loss: 2.1937 - acc: 0.131 - ETA: 1:07 - loss: 2.1909 - acc: 0.140 - ETA: 1:01 - loss: 2.1920 - acc: 0.133 - ETA: 56s - loss: 2.1907 - acc: 0.140 - ETA: 52s - loss: 2.1875 - acc: 0.13 - ETA: 49s - loss: 2.1854 - acc: 0.14 - ETA: 46s - loss: 2.1830 - acc: 0.14 - ETA: 45s - loss: 2.1766 - acc: 0.14 - ETA: 42s - loss: 2.1649 - acc: 0.15 - ETA: 41s - loss: 2.1709 - acc: 0.14 - ETA: 39s - loss: 2.1726 - acc: 0.15 - ETA: 38s - loss: 2.1726 - acc: 0.14 - ETA: 36s - loss: 2.1692 - acc: 0.15 - ETA: 35s - loss: 2.1647 - acc: 0.15 - ETA: 34s - loss: 2.1653 - acc: 0.15 - ETA: 33s - loss: 2.1645 - acc: 0.14 - ETA: 32s - loss: 2.1622 - acc: 0.15 - ETA: 31s - loss: 2.1567 - acc: 0.15 - ETA: 30s - loss:

2592/2592 [==============================] - ETA: 29s - loss: 1.8717 - acc: 0.28 - ETA: 29s - loss: 1.8839 - acc: 0.26 - ETA: 30s - loss: 1.9275 - acc: 0.26 - ETA: 29s - loss: 1.9262 - acc: 0.28 - ETA: 28s - loss: 1.9171 - acc: 0.28 - ETA: 28s - loss: 1.9040 - acc: 0.32 - ETA: 28s - loss: 1.9133 - acc: 0.32 - ETA: 28s - loss: 1.9271 - acc: 0.33 - ETA: 27s - loss: 1.9399 - acc: 0.31 - ETA: 27s - loss: 1.9330 - acc: 0.32 - ETA: 28s - loss: 1.9467 - acc: 0.32 - ETA: 27s - loss: 1.9485 - acc: 0.32 - ETA: 27s - loss: 1.9446 - acc: 0.32 - ETA: 26s - loss: 1.9477 - acc: 0.33 - ETA: 26s - loss: 1.9701 - acc: 0.31 - ETA: 25s - loss: 1.9627 - acc: 0.31 - ETA: 25s - loss: 1.9583 - acc: 0.31 - ETA: 25s - loss: 1.9557 - acc: 0.31 - ETA: 24s - loss: 1.9540 - acc: 0.31 - ETA: 24s - loss: 1.9579 - acc: 0.31 - ETA: 24s - loss: 1.9575 - acc: 0.30 - ETA: 23s - loss: 1.9546 - acc: 0.31 - ETA: 23s - loss: 1.9523 - acc: 0.31 - ETA: 22s - loss: 1.9461 - acc: 0.31 - ETA: 22s - loss: 1.9414 - acc: 0.31 - ETA: 

2592/2592 [==============================] - ETA: 28s - loss: 1.5838 - acc: 0.37 - ETA: 28s - loss: 1.5072 - acc: 0.43 - ETA: 29s - loss: 1.5494 - acc: 0.45 - ETA: 28s - loss: 1.5814 - acc: 0.46 - ETA: 29s - loss: 1.5913 - acc: 0.43 - ETA: 28s - loss: 1.5833 - acc: 0.42 - ETA: 28s - loss: 1.5748 - acc: 0.42 - ETA: 27s - loss: 1.5993 - acc: 0.39 - ETA: 27s - loss: 1.6064 - acc: 0.39 - ETA: 27s - loss: 1.5894 - acc: 0.40 - ETA: 27s - loss: 1.5783 - acc: 0.41 - ETA: 26s - loss: 1.5685 - acc: 0.42 - ETA: 26s - loss: 1.5722 - acc: 0.43 - ETA: 25s - loss: 1.5530 - acc: 0.43 - ETA: 25s - loss: 1.5509 - acc: 0.44 - ETA: 25s - loss: 1.5539 - acc: 0.44 - ETA: 24s - loss: 1.5639 - acc: 0.44 - ETA: 24s - loss: 1.5649 - acc: 0.44 - ETA: 23s - loss: 1.5601 - acc: 0.44 - ETA: 23s - loss: 1.5706 - acc: 0.43 - ETA: 23s - loss: 1.5727 - acc: 0.43 - ETA: 22s - loss: 1.5658 - acc: 0.43 - ETA: 22s - loss: 1.5649 - acc: 0.43 - ETA: 21s - loss: 1.5784 - acc: 0.43 - ETA: 21s - loss: 1.5762 - acc: 0.43 - ETA: 

2592/2592 [==============================] - ETA: 33s - loss: 1.4635 - acc: 0.43 - ETA: 30s - loss: 1.4191 - acc: 0.45 - ETA: 29s - loss: 1.3582 - acc: 0.46 - ETA: 30s - loss: 1.4400 - acc: 0.46 - ETA: 29s - loss: 1.4161 - acc: 0.46 - ETA: 28s - loss: 1.3955 - acc: 0.46 - ETA: 28s - loss: 1.3515 - acc: 0.49 - ETA: 27s - loss: 1.3458 - acc: 0.49 - ETA: 27s - loss: 1.3706 - acc: 0.48 - ETA: 27s - loss: 1.3861 - acc: 0.47 - ETA: 26s - loss: 1.3907 - acc: 0.47 - ETA: 26s - loss: 1.4086 - acc: 0.47 - ETA: 26s - loss: 1.3890 - acc: 0.47 - ETA: 25s - loss: 1.3759 - acc: 0.48 - ETA: 25s - loss: 1.3643 - acc: 0.49 - ETA: 24s - loss: 1.3621 - acc: 0.49 - ETA: 24s - loss: 1.3532 - acc: 0.49 - ETA: 24s - loss: 1.3499 - acc: 0.49 - ETA: 23s - loss: 1.3568 - acc: 0.49 - ETA: 23s - loss: 1.3520 - acc: 0.49 - ETA: 22s - loss: 1.3567 - acc: 0.48 - ETA: 22s - loss: 1.3656 - acc: 0.48 - ETA: 22s - loss: 1.3687 - acc: 0.48 - ETA: 21s - loss: 1.3807 - acc: 0.47 - ETA: 21s - loss: 1.3875 - acc: 0.47 - ETA: 

2592/2592 [==============================] - ETA: 28s - loss: 0.9920 - acc: 0.71 - ETA: 28s - loss: 1.0311 - acc: 0.62 - ETA: 29s - loss: 1.0823 - acc: 0.60 - ETA: 29s - loss: 1.1160 - acc: 0.58 - ETA: 29s - loss: 1.1546 - acc: 0.56 - ETA: 28s - loss: 1.1589 - acc: 0.57 - ETA: 28s - loss: 1.1624 - acc: 0.56 - ETA: 28s - loss: 1.1760 - acc: 0.55 - ETA: 27s - loss: 1.1763 - acc: 0.55 - ETA: 27s - loss: 1.1552 - acc: 0.56 - ETA: 26s - loss: 1.1550 - acc: 0.55 - ETA: 26s - loss: 1.1449 - acc: 0.56 - ETA: 26s - loss: 1.1255 - acc: 0.57 - ETA: 25s - loss: 1.1336 - acc: 0.56 - ETA: 25s - loss: 1.1562 - acc: 0.56 - ETA: 25s - loss: 1.1778 - acc: 0.55 - ETA: 24s - loss: 1.1879 - acc: 0.55 - ETA: 24s - loss: 1.1855 - acc: 0.56 - ETA: 23s - loss: 1.1769 - acc: 0.57 - ETA: 23s - loss: 1.1722 - acc: 0.57 - ETA: 23s - loss: 1.1730 - acc: 0.56 - ETA: 22s - loss: 1.1733 - acc: 0.56 - ETA: 22s - loss: 1.1867 - acc: 0.55 - ETA: 22s - loss: 1.1851 - acc: 0.55 - ETA: 21s - loss: 1.1954 - acc: 0.55 - ETA: 

...

Predicted emotion:  Optimism
Actual emotion   :  Disapproval



Louis was reconstituting the royal family, keeping his adult sons on tenterhooks.

Predicted emotion:  Optimism
Actual emotion   :  Aggression



Louis was reconstituting the royal family, keeping his adult sons on tenterhooks.

Predicted emotion:  Optimism
Actual emotion   :  Optimism



McAllister turned off the gas ring, blew out the match, and walked to the front door, grumbling to herself, hold your horses, I'm coming, I'm coming, when another urgent series of knocks sounded.

Predicted emotion:  Optimism
Actual emotion   :  Aggression



McAllister turned off the gas ring, blew out the match, and walked to the front door, grumbling to herself, hold your horses, I'm coming, I'm coming, when another urgent series of knocks sounded.

Predicted emotion:  Optimism
Actual emotion   :  Optimism



I've kept myself fit and I was raring to go.

Predicted emotion:  Aggression
Actual emotion   :  Aggression



I've kept myself 

Actual emotion   :  Optimism



If it takes off in the States, the sky 's the limit.

Predicted emotion:  Optimism
Actual emotion   :  Optimism



Theda's jaw dropped perceptibly, and she was conscious of a sliver of tender warmth slipping into her heart.

Predicted emotion:  Awe
Actual emotion   :  Awe



Theda's jaw dropped perceptibly, and she was conscious of a sliver of tender warmth slipping into her heart.

Predicted emotion:  Awe
Actual emotion   :  Disapproval



So startled was he by this sudden onslaught, Ryker momentarily froze, rooted to the spot

Predicted emotion:  Awe
Actual emotion   :  Awe



So startled was he by this sudden onslaught, Ryker momentarily froze, rooted to the spot

Predicted emotion:  Awe
Actual emotion   :  Disapproval



Rachel's son Alexander, struck dumb with terror after he saw her murdered, is three tomorrow.

Predicted emotion:  Submission
Actual emotion   :  Neutral



Two men had come to blows, an arm had been broken.

Predicted emotion:  Awe


The justification for doing the work might be seen to lie in the material benefits it produced.

Predicted emotion:  Disapproval
Actual emotion   :  Optimism



Something hit me on the back of the head, here, and I went out like a light.

Predicted emotion:  Neutral
Actual emotion   :  Awe



Something hit me on the back of the head, here, and I went out like a light.

Predicted emotion:  Neutral
Actual emotion   :  Disapproval



Of course people are talking pretty big at the moment but half of them are as ready to drop as I am.

Predicted emotion:  Neutral
Actual emotion   :  Submission



A development which seems likely to bear fruit was the creation of a working party from within the panel.

Predicted emotion:  Neutral
Actual emotion   :  Optimism



Competition climbing is laughable in this country, and is probably a dead duck anyway.

Predicted emotion:  Disapproval
Actual emotion   :  Disapproval



Yet there are certain composers who fall flat on their face unless the adrenali

Actual emotion   :  Submission



Still, the Poles appear determined to make a go of it.

Predicted emotion:  Optimism
Actual emotion   :  Optimism



But if Mr Kohl is now riding high, East German expectations are riding higher.

Predicted emotion:  Aggression
Actual emotion   :  Aggression



But if Mr Kohl is now riding high, East German expectations are riding higher.

Predicted emotion:  Aggression
Actual emotion   :  Optimism



Not since the surprise smash hit of the year cast a rosy glow over Shore's vehicle.

Predicted emotion:  Neutral
Actual emotion   :  Aggression



Not since the surprise smash hit of the year cast a rosy glow over Shore's vehicle.

Predicted emotion:  Neutral
Actual emotion   :  Optimism



All in all, it can only mean that tea time is on the up and up.

Predicted emotion:  Optimism
Actual emotion   :  Optimism



Ronni's jaw dropped open.

Predicted emotion:  Disapproval
Actual emotion   :  Love



They'll come to blows with the Kurds, who'll want their 

It may not be the London Palladium but it is a smash hit.

Predicted emotion:  Neutral
Actual emotion   :  Love



Virginia's jaw dropped open in surprise.

Predicted emotion:  Disapproval
Actual emotion   :  Awe



Virginia's jaw dropped open in surprise.

Predicted emotion:  Disapproval
Actual emotion   :  Disapproval



Alain was getting quite hot under the collar and Dieter seemed to be winding him up on purpose.

Predicted emotion:  Contempt
Actual emotion   :  Contempt



Alain was getting quite hot under the collar and Dieter seemed to be winding him up on purpose.

Predicted emotion:  Contempt
Actual emotion   :  Aggression



I'll thank you to mind your own business, Mrs S.

Predicted emotion:  Neutral
Actual emotion   :  Contempt



I'll thank you to mind your own business, Mrs S.

Predicted emotion:  Neutral
Actual emotion   :  Aggression



A jogger out for his early morning run passed a lone car parked by the side of the road, its windows all steamed up.

Predicted emotion